In [ ]:
import sys
project_path = r"D:/DaiHoc/machinelearning/TLCN/DoAnTotNghiep_chat_bot/"
sys.path.append(project_path)
from source.function.utils_result import RAG
from source.search.utils_search import Qdrant_Utils
from source.rerank.utils_rerank import Rerank_Utils  
from source.model.embedding_model import Sentences_Transformer_Embedding
from source.model.extract_model import Bert_Extract
from source.model.generate_model import Gemini
from source.model.rerank_model import Cohere
from source.data.vectordb.qdrant import Qdrant_Vector
from source.core.config import Settings
from source.model.rerank_model_finetune import RerankModelFinetune
from source.generate.generate import Gemini_Generate
from source.extract.utils_extract import Extract_Information
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm
setting=Settings()
gemini=Gemini(setting)
print(setting.RERANK)
cohere_api=Cohere(setting)
bert=Bert_Extract(setting)
rerank_model_fintuned=RerankModelFinetune(setting)
sentences_transformer_embedding=Sentences_Transformer_Embedding(setting)
qdrant=Qdrant_Vector(setting,sentences_transformer_embedding)
rerank_Utils=Rerank_Utils(cohere_api,rerank_model_fintuned)
extract_Utils= Extract_Information(bert)
generate_Utils=Gemini_Generate(gemini,setting)
qdrant_Utils=Qdrant_Utils(qdrant, generate_Utils)
rag=RAG(generate_Utils,extract_Utils,qdrant_Utils,rerank_Utils,setting,sentences_transformer_embedding)
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai 
from sklearn.metrics.pairwise import cosine_similarity
import os

hghaan/rerank_model


## 1.Eval Gemini model

In [2]:
# Đọc file CSV chứa câu hỏi và câu trả lời đã có (nếu có)
df = pd.read_csv('./data/data_processed/final_data_system_response.csv')

In [ ]:
print(rerank_Utils)

: 

In [ ]:
def Generate_Response(df_batch: pd.DataFrame) -> list[str]:
    """
    Sinh câu trả lời cho mỗi câu hỏi trong batch.
    Trả về danh sách answer tương ứng.
    """
    answers = []
    for question in tqdm(df_batch['question'], desc="Sinh trả lời"):    
        article_doc = rag.get_gemini_response_rag_final(question)
        answers.append(article_doc)
    return answers

df['answer_from_gemini_rag_final'] = Generate_Response(df)


Sinh trả lời:   0%|          | 0/1204 [00:00<?, ?it/s]

['Câu 1. Tôi có thể kết hôn với cháu ruột của thím mình không?', 'Tôi có được phép kết hôn với người có quan hệ huyết thống là cháu ruột của thím theo quy định pháp luật không?', 'Quy định pháp luật Việt Nam về việc kết hôn giữa những người có quan hệ họ hàng, cụ thể là cháu ruột của thím, được phép hay không?', 'Điều kiện kết hôn theo Luật Hôn nhân và Gia đình có cho phép kết hôn giữa cháu ruột và thím không?']
None
Query 1 có 25 kết quả
None
Query 2 có 25 kết quả
None
Query 3 có 25 kết quả
should=[FieldCondition(key='metadata.LoaiVanBanFilter', match=MatchValue(value='luật'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None, is_empty=None, is_null=None)] min_should=None must=None must_not=None
Query 4 có 25 kết quả
Tổng số kết quả tìm được: 100
Số document khi xoá trùng 38
Số document sau khi qua rerank: 5


Sinh trả lời:   0%|          | 1/1204 [00:48<16:08:27, 48.30s/it]

['Câu 2. Anh J và chị O học tiểu học cùng nhau, anh J theo bố mẹ sang định cư tại Đan Mạch. Khi về thăm quê, anh J có gặp lại chị O, từ đó cả hai nối lại tình bạn. Sau một thời gian trao đổi, liên hệ với nhau qua điện thoại, mạng xã hội, chị O tỏ ý muốn sang định cư tại Đan Mạch và nhờ anh J giúp đỡ bằng cách đồng ý kết hôn với chị. Hai bên sẽ ly hôn sau khi chị O được nhập quốc tịch và đã sang cư trú tại Đan Mạch. Trưởng hợp này pháp luật có nghiêm cấm không và nếu J và O vẫn thực hiện thì xử lý như thế nào?', 'Hành vi kết hôn giả tạo để nhập quốc tịch có bị cấm theo quy định pháp luật Việt Nam không và chế tài xử lý nếu vi phạm?', 'Hôn nhân không thực chất nhằm mục đích định cư nước ngoài, cụ thể là nhập quốc tịch Đan Mạch, có hợp pháp theo luật Việt Nam và hậu quả pháp lý của việc này là gì?', 'Pháp luật Việt Nam có điều khoản nào ngăn cấm việc kết hôn chỉ để nhập quốc tịch nước ngoài (Đan Mạch) và nếu vẫn thực hiện thì J và O sẽ bị xử phạt như thế nào?']
None
Query 1 có 25 kết quả


Sinh trả lời:   0%|          | 2/1204 [02:00<20:52:59, 62.55s/it]

...

['Xin hỏi, hợp đồng lao động vô hiệu trong những trường hợp nào? Cơ quan có thẩm quyền tuyên bố hợp đồng vô hiệu', 'Các trường hợp hợp đồng lao động bị tuyên bố vô hiệu theo quy định pháp luật hiện hành và cơ quan nào có thẩm quyền?', 'Hợp đồng lao động vô hiệu khi nào? Quy định về thẩm quyền tuyên bố hợp đồng lao động vô hiệu?', 'Hậu quả pháp lý của hợp đồng lao động vô hiệu và cơ quan có thẩm quyền giải quyết tranh chấp hợp đồng lao động vô hiệu?']
None
Query 1 có 25 kết quả
None
Query 2 có 25 kết quả
None
Query 3 có 25 kết quả
None
Query 4 có 25 kết quả
Tổng số kết quả tìm được: 100
Số document khi xoá trùng 43
Số document sau khi qua rerank: 5


Sinh trả lời:  19%|█▉        | 229/1204 [4:03:04<17:25:54, 64.36s/it]

['Cho thuê lại lao động là gì? Nguyên tắc hoạt động cho thuê lại lao động?', 'Cho thuê lại lao động được định nghĩa như thế nào theo quy định pháp luật hiện hành? Nguyên tắc hoạt động của dịch vụ cho thuê lại lao động được quy định cụ thể ra sao?', 'Quy định pháp luật về cho thuê lại lao động, bao gồm khái niệm và các nguyên tắc hoạt động cơ bản của dịch vụ này?', 'Dịch vụ cho thuê lại lao động là gì? Các quy định và nguyên tắc chi phối hoạt động cho thuê lại lao động được pháp luật Việt Nam quy định như thế nào?']
None
Query 1 có 25 kết quả
None
Query 2 có 25 kết quả
None
Query 3 có 25 kết quả
None
Query 4 có 25 kết quả
Tổng số kết quả tìm được: 100
Số document khi xoá trùng 38
Số document sau khi qua rerank: 5


Sinh trả lời:  19%|█▉        | 230/1204 [4:03:57<16:29:32, 60.96s/it]

['Doanh nghiệp cho thuê lại lao động có quyền và nghĩa vụ gì?', 'Quyền và nghĩa vụ pháp lý của doanh nghiệp cho thuê lại lao động được quy định như thế nào?', 'Doanh nghiệp hoạt động trong lĩnh vực cho thuê lại lao động có những quyền hạn và trách nhiệm gì theo luật định?', 'Nghĩa vụ và quyền của doanh nghiệp cho thuê lại lao động được pháp luật Việt Nam quy định cụ thể ra sao?']
None
Query 1 có 25 kết quả
None
Query 2 có 25 kết quả
None
Query 3 có 25 kết quả
None
Query 4 có 25 kết quả
Tổng số kết quả tìm được: 100
Số document khi xoá trùng 33


In [ ]:
final_output_path = './data/data_processed/final_data_system_response.csv'
df.to_csv(final_output_path, index=False)
print(f" Đã cập nhật và lưu toàn bộ vào: {final_output_path}")

> Configure

In [ ]:
setting=Settings()

> Get embedding gemini 

In [ ]:
import google.generativeai as genai

genai.configure(api_key=gemini.key_manager.get_next_key())

def get_gemini_embedding(texts):
    try :
        result = genai.embed_content(
            model="models/text-embedding-004",
            content=texts,
            task_type="SEMANTIC_SIMILARITY"
        )
        
        return result['embedding']
    except Exception as e:
        print("Đã xảy ra lỗi:", e)
        return [0.0] * 768

> get embedding cohere 

In [ ]:
# import cohere

# def get_cohere_embedding(text: str, model_name: str = "embed-multilingual-v3.0") -> list:
#     try:
#         co = cohere.ClientV2(api_key=cohere_api.key_manager.get_next_key())
#         res = co.embed(
#             texts=[text],
#             model=model_name,
#             input_type="classification",
#             embedding_types=["float"],
#         )a

#         # Trả về nhúng đầu tiên
#         return res.embeddings.float[0]
#     except Exception as e:
#         print("Đã xảy ra lỗi:", e)
#         return [0.0] * 1024  # Trả về danh sách 1024 số 0 nếu có lỗi


>Get  rouge-L & F1 Score

In [ ]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def compute_rouge_l(pred: str, ref: str) -> float:
    try:
        score = scorer.score(ref, pred)
        return score['rougeL'].fmeasure
    except:
        return 0.0

def compute_f1(pred: str, ref: str) -> float:
    pred_tokens = pred.lower().split()
    ref_tokens = ref.lower().split()
    common = set(pred_tokens) & set(ref_tokens)
    if not pred_tokens or not ref_tokens:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

In [ ]:
df = pd.read_csv('./data/data_processed/final_data_system_response.csv')
df = df.dropna(subset=['answer', 'answer_from_gemini_rag_final'])

print(df.shape)

In [ ]:

pattern = r'Xin\s+lỗi\s+bạn'  # regex cho “Xin lỗi bạn” (có thể có nhiều khoảng trắng)

# Chỉ giữ lại những dòng không khớp pattern
df = df[~df['answer_from_gemini_rag_basic'].str.contains(pattern, regex=True, na=False)]

In [ ]:
# # df['answer_embedding_cohere'] = df['answer'].apply(get_cohere_embedding)
# df['answer_from_gemini_embedding_cohere'] = df['answer_from_gemini'].apply(get_cohere_embedding)


df['answer_embedding'] = df['answer'].apply(get_gemini_embedding)
df['answer_from_gemini_rag_final_embedding'] = df['answer_from_gemini_rag_final'].apply(get_gemini_embedding)

df['cosine_similarity_gemini_rag_final'] = df.apply(
    lambda row: cosine_similarity(
        [row['answer_embedding']], 
        [row['answer_from_gemini_rag_final_embedding']]
    )[0][0],
    axis=1
)

# df['cosine_similarity_cohere'] = df.apply(
#     lambda row: cosine_similarity(
#         [row['answer_embedding_cohere']], 
#         [row['answer_from_gemini_embedding_cohere']]
#     )[0][0],
#     axis=1
# )

In [ ]:
df['rouge_l'] = df.apply(lambda row: compute_rouge_l(row['answer_from_gemini_rag_final'], row['answer']), axis=1)
df['f1'] = df.apply(lambda row: compute_f1(row['answer_from_gemini_rag_final'], row['answer']), axis=1)


In [ ]:
print(f" Trung bình Cosine (Gemini): {df['cosine_similarity_gemini_rag_final'].mean():.4f}")
print(f" Trung bình ROUGE-L: {df['rouge_l'].mean():.4f}")
print(f" Trung bình F1 Score: {df['f1'].mean():.4f}") 